### step:1  Load the pdf documnets 

In [91]:
# imoprt liabrary 

import langchain 
import openai 
import pinecone  

from langchain.document_loaders import PyPDFDirectoryLoader          # it can load any kind of doucment or pdf 
from langchain.text_splitter import RecursiveCharacterTextSplitter     # here it spit the text int chunks 
from langchain.embeddings.openai import OpenAIEmbeddings                # now taht chunks convert it into vectore 
from langchain.vectorstores import Pinecone                             # our vector stores in pinecone 
from langchain.llms import OpenAI


In [70]:
import os 
from dotenv import load_dotenv 
load_dotenv()  

True

In [71]:
# read the pdf file document  

def read_doc(directory) : 
    file_loader= PyPDFDirectoryLoader(directory) 
    documents = file_loader.load() 
    return documents 

In [72]:
doc = read_doc("pdf_documents/") 

doc 

[Document(page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023', metadata={'source': 'pdf_documents\\budget_speech.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': 'pdf_documents\\budget_speech.pdf', 'page': 1}),
 Document(page_content='CONTENTS \nPART-A \n Page No.  \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal  – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector  \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management 24 \nPART B  \n  \nIndirect Taxes  27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Prec

### step:2  convert the doc data into text chunks 

In [73]:
def chunk_data(docs , chunk_size=700 , chunk_overlap=50) : 
    text_split=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap) 
    doc= text_split.split_documents(docs) 
    return doc 
    

In [74]:
documents =chunk_data(docs=doc) 
documents

[Document(page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023', metadata={'source': 'pdf_documents\\budget_speech.pdf', 'page': 0}),
 Document(page_content='CONTENTS \nPART-A \n Page No.  \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal  – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector  \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management 24 \nPART B  \n  \nIndirect Taxes  27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7 Metals  \n\uf0b7 Compounded Rubber  \n\uf0b7 Cigarettes  \n  \nDirect Taxe

In [75]:
len(documents)

165

### step:3 Convert the Document into Vector by using OpenaiEmbedding

In [76]:
embeddings = OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"])

In [18]:
vectors =embeddings.embed_query(" hi good morning")

In [20]:
print(vectors)

print(len(vectors))   # <-- when we create our databse we have to specify that length , so its imp 

[-0.018963978750378713, 0.01572653770597149, -0.020972983019387284, -0.030659711546702238, -0.008963742843398683, 0.01910473697567272, -0.02326350373898387, -0.00463862511153188, -0.0034357815450561423, 0.021753550512997197, 0.014127011741214114, -0.004686610573824939, -0.030941227997290252, -0.03690425894616184, -0.0023321086945661314, 0.0033686012924862116, 0.04542653441887733, -0.0057742881926070525, 0.005732700450468138, -0.01380710710705616, -0.02317393100021316, -0.01333364694465084, 0.00397002307572408, -0.022981987288395855, 0.008387913570591831, -0.0052464448477545274, 0.02421042220114944, -0.03667392798209712, 0.0005778287058741685, -0.009968244977902968, -0.017313267299098038, -0.007946444337263573, -0.02264928535128454, 0.0028039688448563316, -0.012821799529998113, -0.018874404148962937, -0.012828196784490991, -0.0041939567852959155, 0.00044626769806684204, 0.009577960299775475, 0.009654737598237892, -0.027818952434915382, 0.009373221147649878, -0.015944074161050446, -0.023

#### step : 4 Create Our Vectorserchdb in pinecone 

In [82]:
from pinecone import Pinecone, create_index, ServerlessSpec

# Create an instance of the Pinecone class
pinecone = Pinecone(api_key="752e0a47-f03b-48b4-b9d1-0e9ef65e76c7", environment="Serverless")

index_name = "langchainvectorsearch1"

In [92]:
index=Pinecone.from_documents(doc,embeddings,index_name)

In [95]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [94]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI 

llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

e:\GenAi_proj\venv2\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [96]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

##### Answer is :
 The government is promoting cooperative-based economic development models and investing in initiatives such as the Agriculture Accelerator Fund and Atmanirbhar Clean Plant Program to help farmers, especially small and marginal farmers, and other marginalised sections.

In [37]:
# from pinecone.grpc import PineconeGRPC as Pinecone

# pc = Pinecone(api_key="752e0a47-f03b-48b4-b9d1-0e9ef65e76c7")

# pc.describe_index("serverless-index")

# Response:
# {'dimension': 1536,
#  'host': 'serverless-index-4zo0ijk.svc.us-weset2-aws.pinecone.io',
#  'metric': 'cosine',
#  'name': 'serverless-index',
#  'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
#  'status': {'ready': True, 'state': 'Ready'}}


In [93]:
# from pinecone import Pinecone, ServerlessSpec

# pinecone = Pinecone(api_key="752e0a47-f03b-48b4-b9d1-0e9ef65e76c7")

# try:
#     index_description = pinecone.describe_index("langchainpinecone")
#     print("Index 'langchainpinecone' already exists.")
# except Exception:
#     pinecone.create_index(
#         name="langchainpinecone",
#         dimension=1536,  # Replace with your model dimensions
#         metric="cosine",  # Replace with your model metric
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         )
#     )
#     print("Index 'langchainpinecone' created successfully.")